<h1>Import libraries</h1>

<h1>Establish connection to snowflake and Importing Tables</h1>

In [1]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = 'SOL_ASSORTMENT_PLANNING_COMB'
 
sf_df = my_session.sql("select * from {}".format(table_name))
sales_bangalore_2022 = sf_df.to_pandas()
 
sales_bangalore_2022.head()

,TRANSACTION_DATE,MNTH_CODE,YEAR_CODE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,DISTRIBUTOR_CODE,PRODUCT_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,CATEGORY,VARIANT,BRAND
0,2023-08-01,202204,2022,142.84,16,0.000272,142.857143,DB0110,PRD0028,OL97172,Barrington,Rhode Island,Dolphin,Str3,Soap,Toilet Soap,Indigo
1,2023-08-01,202204,2022,336.36,2,0.000950,336.363636,DB0652,PRD0098,OL97443,Windsor,Connecticut,Dolphin,Str2,Lotion,Head Lotion,Beige
2,2023-08-01,202204,2022,109.09,2,0.000220,109.090909,DB0110,PRD0015,OL96862,Bedford,Ohio,Scott,Str1,Kids Care,Diapers,Pearl
3,2023-08-01,202204,2022,285.69,32,0.000800,285.714286,DB0655,PRD0106,OL96508,Great Falls,Montana,Scott,Str3,Hair Care,Shampoo,Hair & Shoulders
4,2023-08-01,202204,2022,172.73,1,0.000375,172.727273,DB0209,PRD0093,OL96679,Charlevoix,Michigan,City Center,Str2,Soap,Medicated Soap,Amber


In [2]:
table_name = 'SOL_ASSORTMENT_PLANNING_TRAIN_DATA_MODEL'
 
sf_df = my_session.sql("select * from {}".format(table_name))
train_final = sf_df.to_pandas()
 
train_final.head()

,S_NO,UNIQUE_ID,FREQUENCYM,SALES_VALUE_AVG,SALES_UNITS_AVG,SALES_INDICATOR,SCHEME_AMOUNT_PERPRODUCT
0,0,PRD0002-OL10330,0.6,0.004323138,0.00040404,1,0.000465
1,1,PRD0002-OL10346,0.2,0.001429593,0.00013468,0,0.000465
2,2,PRD0002-OL10347,0.2,0.001766519,0.00016835,0,0.000465
3,3,PRD0002-OL10358,1.0,0.01577123,0.001481481,1,0.000465
4,4,PRD0002-OL10386,0.8,0.00841699,0.000791246,1,0.000465


In [ ]:
table_name = 'SOL_ASSORTMENT_PLANNING_TEST_SET_MODEL'
 
sf_df = my_session.sql("select * from {}".format(table_name))
test_final = sf_df.to_pandas()
 
test_final.head()

<h1>Analyze data</h1>

In [ ]:
sales_bangalore_2022 = session.table("SOL_ASSORTMENT_PLANNING_COMB").to_pandas()

In [ ]:
sales_bangalore_2022.head()

In [3]:
train_final = session.table("SOL_ASSORTMENT_PLANNING_TRAIN_DATA_MODEL").to_pandas()
train_final.head(1)

NameError: name 'session' is not defined

In [ ]:
table_name = 'SOL_ASSORTMENT_PLANNING_COMB'
 
sf_df = my_session.sql("select * from {}".format(table_name))
sales_bangalore_2022 = sf_df.to_pandas()
 
sales_bangalore_2022.head()

In [ ]:
train_final.columns = train_final.columns.str.lower()

In [ ]:
train_final.head(1)

In [ ]:
master_data = sales_bangalore_2022.copy()
master_data.head(1)

In [ ]:
master_data.columns = master_data.columns.str.lower()
master_data.head(1)

In [ ]:
df1=master_data.copy
master_data['unique_id']=master_data['product_code']+'-'+ master_data['outlet_code']

<h1>Train Model</h1>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_final[['frequencym','sales_value_avg','scheme_amount_perproduct']],train_final['sales_indicator'],test_size=0.3,random_state=42)

In [ ]:
import pickle

with open('logreg.pkl', 'rb') as model_file:
   loaded_model = pickle.load(model_file)
probs = loaded_model.predict_proba(X_test)
probs

In [ ]:
cutoff = 0.3
y_pred = (probs[:,1]>=cutoff).astype(int)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

<h1> Test model </h1>

In [ ]:
test_final = session.table("SOL_ASSORTMENT_PLANNING_TEST_SET_MODEL").to_pandas()
test_final.columns = test_final.columns.str.lower()
test_final.head(1)


In [ ]:
X_test_unseen = test_final[['frequencym','sales_value_avg','scheme_amount_perproduct']]
y_test_unseen = test_final['sales_indicator']
print(X_test_unseen.shape,y_test_unseen.shape)
probs = loaded_model.predict_proba(X_test_unseen)
test_final['probablity']=probs[:,1]
y_pred = (probs[:,1]>=0.4).astype(int)
test_final

In [ ]:
test_final['prediction']= test_final['probablity'].apply(lambda x: 1  if x>=0.3 else 0)
test_final

In [ ]:
test_final_static = test_final.copy()
print(classification_report(test_final['sales_indicator'],test_final['prediction']))

In [ ]:
test_final.reset_index(drop=True)
test_final['OOS'] = test_final['probablity'].apply(lambda x:1 if x>0.3 else 0)

In [ ]:
soq_master = master_data.groupby('unique_id').agg({'sales_units':'sum','mnth_code':'nunique'})
soq_master['SOQ'] = soq_master['sales_units']/soq_master['mnth_code']
soq_master.reset_index()
final_reco = pd.merge(soq_master,test_final,on='unique_id',how='inner')

In [ ]:
final_reco = final_reco[['unique_id','prediction','SOQ','OOS']]
final_reco.head()

In [ ]:
ms_reco = final_reco.copy()
ms_reco.columns = ['unique_id', 'ms_flag', 'soq', 'oos_flag']

<h1> Write output to a snowflake table </h1>

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
ms_reco[['product', 'outlet']] = ms_reco['unique_id'].str.split('-', expand=True)
ms_reco.head()

In [ ]:
ms_reco=session.createDataFrame(
        ms_reco.values.tolist(),
        schema=ms_reco.columns.tolist())

In [ ]:

ms_reco.write.mode("overwrite").save_as_table("SOL_CPG_DB.SOL_SALES_SCHEMA.SOL_ASSORTMENT_PREDICTION_TABLE")

In [ ]:
req = requests.Request()
req.json = {"payload":X_test_unseen.head(1).to_json()}
print({'payload': req.json})

In [ ]:
from fosforml import *

@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.DataFrame(model.predict(data))
    return prediction.to_json()

In [ ]:
print(score(loaded_model, req))

<h1>Register model in FDC</h1>

In [ ]:
from fosforml.constants import MLModelFlavours


In [ ]:
tmp = register_model(loaded_model, 
               score, 
               name="Assortment_Planning_Prediction", 
               description="prediction of assortment across retailers by product",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               y_true=y_test,
               y_pred=y_pred, 
               prob=probs, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True, 
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['No Failure',' or Failure'],
               kyd=True, kyd_score = True)